# Setup Variables

MNIST, FashionMNIST, GTSRB, Cifar10

In [12]:
DATASET = 'FashionMNIST'
POSTFIX = 'Adam-32-60'
FILENAME_POSTFIX = f'{DATASET}_{POSTFIX}'
SEED = 42
CUDA = 0
GPU_NAME = f'cuda:{CUDA}'

In [13]:
import os
from pathlib import Path

base = Path().cwd()

if base.name != 'runtime-monitoring':
    os.chdir('../')
    base = Path().cwd()

base

PosixPath('/home/ah19/runtime-monitoring')

In [14]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

# Libraries

In [15]:
import pandas as pd
import numpy as np
from pathlib import Path

from fastprogress import progress_bar

from utilities.utils import load_json, load_pickle
from utilities.pathManager import fetchPaths
from utilities.MonitorUnifiedBDD import MonitorBDD

# Paths

In [16]:
paths = fetchPaths(base, DATASET, POSTFIX, False)

path_data = paths['data']
path_lhl = paths['lhl']
path_lhl_raw = paths['lhl_raw']
path_lhl_pca = paths['lhl_pca']

# configs = load_json(paths['configuration'])
# config = configs['configuration']
# model_setup = configs['model_setup']
# model_config = configs['model_config']
# optim_name = list(config['optimizer'].keys())[0]
# optim_args = config['optimizer'][optim_name]
# scheduler_name = list(config['scheduler'].keys())[0]
# scheduler_args = config['scheduler'][scheduler_name]

# Seed

In [17]:
np.random.seed(SEED)

# Import Data

In [18]:
FLAVOR = 'raw'

if FLAVOR == 'raw':
    path_lhl = path_lhl_raw
else: path_lhl = path_lhl_pca

# import Data
df_train = pd.read_csv(path_lhl / f"{FILENAME_POSTFIX}_{FLAVOR}_train.csv")

# select only true classified
df_true = df_train[df_train["true"] == True].copy()
df_true = df_true.drop("true", axis=1).reset_index(drop=True)

df_test = pd.read_csv(path_lhl / f"{FILENAME_POSTFIX}_{FLAVOR}_test.csv")

# Shuffel and Split indecies

In [19]:
periods = 5

# shuffle data
schuffled_index = df_true.index.to_numpy().copy()
np.random.shuffle(schuffled_index)

# split data into batches
fragments_index = np.array_split(schuffled_index, periods)

# print shape
schuffled_index.shape, np.concatenate(fragments_index).shape

((57816,), (57816,))

# Build BDD

In [20]:
# each set of data has its own threshold
thld_p = 0.9
eta = 0
neurons = []


# save scores and stats
stats = pd.DataFrame()
train_scores = pd.DataFrame()
test_scores = pd.DataFrame()

# progress bar
pb = progress_bar(range(len(fragments_index)))

# buidl for each fragment
for fragment in pb:
    # indecies
    ix = np.concatenate(fragments_index[0:fragment+1])

    # calculate threshold
    thld = np.quantile( df_true.loc[ix].drop('y', axis=1), thld_p)
    
    # BDD
    patterns = MonitorBDD( df_true.loc[ix].shape[1]-1, thld, neurons=neurons)
    df_train_copy, df_test_copy = patterns.add_dataframe( df_true.loc[ix], eta, eval_dfs=[df_train.loc[ix].copy(), df_test.copy()] )

    
    # score
    df_train_scores = patterns.score_dataframe_multi_eta(df_train_copy, eta)
    df_test_scores = patterns.score_dataframe_multi_eta(df_test_copy, eta)

    
    # add metadata
    patterns.stats['thld'] = f'qth_{thld_p}'
    patterns.stats['period'] = fragment
    patterns.stats['num_observations'] = ix.shape[0]
    df_train_scores['period'] = fragment
    df_train_scores['num_observations'] = ix.shape[0]
    df_test_scores['period'] = fragment
    df_test_scores['num_observations'] = ix.shape[0]
    
    
    # collect scores
    stats = pd.concat([stats, patterns.stats])
    train_scores = pd.concat([train_scores, df_train_scores])
    test_scores = pd.concat([test_scores, df_test_scores])

In [21]:
stats

,thld,eta,build_time,size_mb,reorder_time,num_reorder,num_neurons,start_time,end_time,period,num_observations
1,qth_0.9,0.0,0.029,4306.823,0.0,0.0,60.0,2023-07-02 14:39:27,2023-07-02 14:39:29,0,11564
1,qth_0.9,0.0,0.055,4307.610,0.0,0.0,60.0,2023-07-02 14:39:34,2023-07-02 14:39:38,1,23127
1,qth_0.9,0.0,0.078,4307.815,0.0,0.0,60.0,2023-07-02 14:39:44,2023-07-02 14:39:49,2,34690
1,qth_0.9,0.0,0.103,4308.310,0.0,0.0,60.0,2023-07-02 14:39:57,2023-07-02 14:40:03,3,46253
1,qth_0.9,0.0,0.135,4308.419,0.0,0.0,60.0,2023-07-02 14:40:13,2023-07-02 14:40:21,4,57816


In [22]:
train_scores.loc[ train_scores['y'] == 'all' ]

,y,count,false,false_misclassified,false_classified,outOfPattern,outOfPatternMisclassified,outOfPatternClassified,eta,period,num_observations
10,all,11564.0,1702.0,57.0,1645.0,0.147181,0.033490,0.966510,0,0,11564
10,all,23127.0,1838.0,72.0,1766.0,0.079474,0.039173,0.960827,0,1,23127
10,all,34690.0,1556.0,88.0,1468.0,0.044854,0.056555,0.943445,0,2,34690
10,all,46253.0,975.0,102.0,873.0,0.021080,0.104615,0.895385,0,3,46253
10,all,57816.0,102.0,102.0,0.0,0.001764,1.000000,0.000000,0,4,57816


In [23]:
test_scores.loc[ test_scores['y'] == 'all' ]

,y,count,false,false_misclassified,false_classified,outOfPattern,outOfPatternMisclassified,outOfPatternClassified,eta,period,num_observations
10,all,10000.0,1773.0,100.0,1673.0,0.1773,0.056402,0.943598,0,0,11564
10,all,10000.0,1280.0,70.0,1210.0,0.1280,0.054688,0.945312,0,1,23127
10,all,10000.0,1030.0,54.0,976.0,0.1030,0.052427,0.947573,0,2,34690
10,all,10000.0,879.0,51.0,828.0,0.0879,0.058020,0.941980,0,3,46253
10,all,10000.0,760.0,42.0,718.0,0.0760,0.055263,0.944737,0,4,57816
